In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import matplotlib as mpl
mpl.rcParams["lines.linewidth"] = 0.96
%matplotlib qt

from ppg_fiducials import *

In [42]:
plt.style.use('seaborn-v0_8-whitegrid')

In [2]:
data_path = '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/Data/baselineExcept1st5/'
sub_names = sorted([f for f in os.listdir(data_path) if not f.startswith('.')])

In [3]:
HOM_info_baseline = pd.read_excel('/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/Hom3ostasis_DataCollected.xlsx', header = 1)
ID = HOM_info_baseline["Unnamed: 1"].iloc[0:40].values
sub_ID = [str(i+1) + '_' + ID[i] for i in range(len(ID))]
sub_ID.remove("2_SL")

age = HOM_info_baseline['Età'].iloc[0:40].drop(1)
age.index = sub_ID
AgIx = HOM_info_baseline['AgIx [years]'].iloc[0:40].drop(1)
AgIx.index = sub_ID
SBP = HOM_info_baseline[['Sys [mmHg]']].iloc[0:40].drop(1)
SBP.index = sub_ID
DBP = HOM_info_baseline[['Dia [mmHg]']].iloc[0:40].drop(1)
DBP.index = sub_ID
PWV = HOM_info_baseline['PWV [m/s]'].iloc[0:40].drop(1)
PWV.index = sub_ID
AIx = HOM_info_baseline['AIx@75'].iloc[0:40].drop(1)
AIx.index = sub_ID
sex = HOM_info_baseline['Sesso'].iloc[0:40].drop(1)
sex.index = sub_ID
h = HOM_info_baseline["Altezza [cm]"].iloc[0:40].drop(1).astype(int)
h.index = sub_ID

# LAB

In [4]:
data_path = '/Volumes/Untitled/HOM3ostasis/CinC/Data_lab/baseline/'
sub_names = sorted([f for f in os.listdir(data_path) if not f.startswith('.')])

features_lab = pd.DataFrame()
fiducials_lab = pd.DataFrame()
for i, sub_ID in enumerate(sub_names): 

    fiducials_sub = pd.read_csv(data_path + sub_ID + '/fiducials_norm.csv')
    fiducials_sub['sub_ID'] = sub_ID

    features_sub = pd.read_csv(data_path + sub_ID + '/features_norm.csv')
    features_sub['sub_ID'] = sub_ID

    features_sub["h/dT"] = h[sub_ID] / features_sub["dT"]

    features_lab = pd.concat([features_lab, features_sub], axis = 0)
    fiducials_lab = pd.concat([fiducials_lab, fiducials_sub], axis = 0)

In [5]:
SBP["SBP"] = (SBP["Sys [mmHg]"] > 120).astype(int) # 1 if SBP > 120 mmHg, 0 otherwise
SBP.drop("Sys [mmHg]", axis = 1, inplace = True)

In [6]:
features_lab_median = features_lab.groupby("sub_ID").median()
# merge the SBP information
features_lab_median = features_lab_median.merge(SBP, left_index = True, right_index = True)
features_lab_median.rename(columns = {"SBP": "Hypertension"}, inplace = True)
features_lab_median.loc[features_lab_median["Hypertension"] == 0, "Hypertension"] = "Low BP"
features_lab_median.loc[features_lab_median["Hypertension"] == 1, "Hypertension"] = "High BP"
features_lab_median.rename(columns = {"Hypertension": "SBP"}, inplace = True)

/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_34244/1148257944.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Low BP' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  features_lab_median.loc[features_lab_median["Hypertension"] == 0, "Hypertension"] = "Low BP"


In [7]:
Tsp_med_low = features_lab_median["Tsp"][features_lab_median["SBP"] == "Low BP"].median()
Tsp_sem_low = features_lab_median["Tsp"][features_lab_median["SBP"] == "Low BP"].sem()
Tu_med_low = features_lab_median["Tu"][features_lab_median["SBP"] == "Low BP"].median()
Tu_sem_low = features_lab_median["Tu"][features_lab_median["SBP"] == "Low BP"].sem()
hdT_med_low = features_lab_median["h/dT"][features_lab_median["SBP"] == "Low BP"].median()
hdT_sem_low = features_lab_median["h/dT"][features_lab_median["SBP"] == "Low BP"].sem()
IPA_med_low = features_lab_median["IPA"][features_lab_median["SBP"] == "Low BP"].median()
IPA_sem_low = features_lab_median["IPA"][features_lab_median["SBP"] == "Low BP"].sem()

Tsp_med_high = features_lab_median["Tsp"][features_lab_median["SBP"] == "High BP"].median()
Tsp_sem_high = features_lab_median["Tsp"][features_lab_median["SBP"] == "High BP"].sem()
Tu_med_high = features_lab_median["Tu"][features_lab_median["SBP"] == "High BP"].median()
Tu_sem_high = features_lab_median["Tu"][features_lab_median["SBP"] == "High BP"].sem()
hdT_med_high = features_lab_median["h/dT"][features_lab_median["SBP"] == "High BP"].median()
hdT_sem_high = features_lab_median["h/dT"][features_lab_median["SBP"] == "High BP"].sem()
IPA_med_high = features_lab_median["IPA"][features_lab_median["SBP"] == "High BP"].median()
IPA_sem_high = features_lab_median["IPA"][features_lab_median["SBP"] == "High BP"].sem()

# 24-h

## Basic features

In [8]:
data_path = '/Volumes/Untitled/HOM3ostasis/CinC/Data_24hPulses/mod_newQuality/'
sub_names = sorted([f for f in os.listdir(data_path) if not f.startswith('.')])

features_day = pd.DataFrame()
features_2min30s = pd.DataFrame()
features_hourly = pd.DataFrame()

for i, sub_ID in enumerate(sub_names): 

    features_sub = pd.read_csv(data_path + sub_ID + '/features_basic_ALL.csv')
    features_sub.index = pd.to_datetime(features_sub["start_datetime"])
    features_sub.drop(columns = "start_datetime", inplace = True)
    fiducials = pd.read_csv(data_path + sub_ID + '/fiducials_basic_ALL.csv').drop(columns = "Unnamed: 0")
    fiducials.index = pd.to_datetime(fiducials["start_datetime"])
    fiducials.drop(columns = "start_datetime", inplace = True)
    features_sub["Tu"] = fiducials["u"] - fiducials["onset"]

    features_sub_2min30s = features_sub.resample("2.5 min").median()

    features_sub_hourly = features_sub.resample("h").median()

    # plt.figure(figsize=(15, 10))
    # plt.plot(features_sub_2min30s["Tsp"], label = "Original")
    # plt.plot(features_sub_hourly["Tsp"], label = "Hourly")
    # plt.plot(features_sub["sleep"] * 50)
    # plt.title(sub_ID, fontsize = 20)
    features_sub['sub_ID'] = sub_ID
    features_sub_2min30s["sub_ID"] = sub_ID
    features_sub_hourly["sub_ID"] = sub_ID

    # Add SBP information
    features_sub["SBP"] = SBP.loc[sub_ID].values[0]
    features_sub_2min30s["SBP"] = SBP.loc[sub_ID].values[0]
    features_sub_hourly["SBP"] = SBP.loc[sub_ID].values[0]

    features_day = pd.concat([features_day, features_sub], axis = 0)
    features_2min30s = pd.concat([features_2min30s, features_sub_2min30s], axis = 0)
    features_hourly = pd.concat([features_hourly, features_sub_hourly], axis = 0)

In [76]:
# take the between-subject median of the features for each hour of the day
features_hourly.to_csv('/Volumes/Untitled/HOM3ostasis/CinC/data_spurious/features_hourly.csv')

### Overall

In [71]:
features_hourly['start_datetime'] = pd.to_datetime(features_hourly.index)
features_hourly.reset_index(drop = True, inplace = True)
features_hourly['hour'] = features_hourly['start_datetime'].dt.hour
features_hourly['hour_ordered'] = features_hourly['hour'].apply(lambda x: x if x >= 12 else x + 24)

# Calculate the mean and standard error of Tsp for each hour of the day in the new order
mean_tsp_ordered = features_hourly.groupby('hour_ordered')['Tsp'].mean().sort_index()
sem_tsp_ordered = features_hourly.groupby('hour_ordered')['Tsp'].sem().sort_index()

# Adjust the x-axis labels to match the new order
ordered_hours_labels = [f"{hour % 24}:00" for hour in mean_tsp_ordered.index]


In [72]:
# Plot the mean Tsp with error bars representing the standard error
plt.figure(figsize=(10, 6))
plt.errorbar(range(len(mean_tsp_ordered)), mean_tsp_ordered.values, yerr=sem_tsp_ordered.values, fmt='-o', capsize=5)
plt.xlabel('Hour of the Day')
plt.ylabel('Mean Tsp')
plt.title('Mean Tsp per Hour of the Day with Standard Error (12pm to 12pm)')
plt.grid(True)
plt.xticks(range(len(mean_tsp_ordered)), ordered_hours_labels, rotation=45)
plt.show()

### High vs Low BP

In [16]:
sns.set_context('talk')

In [11]:
basic_features = list(features_hourly.columns[:6]) + ['Tu']
basic_features

['Tsp', 'Tpi', 'Asp', 'AUCpi', 'Tsp/Tpi', 'Tsp/Asp', 'Tu']

In [129]:
# Convert the 'start_datetime' column to datetime format
features_hourly['start_datetime'] = pd.to_datetime(features_hourly.index)
features_hourly.reset_index(drop = True, inplace = True)

# Extract the hour from the 'start_datetime' column
features_hourly['hour'] = features_hourly['start_datetime'].dt.hour

# Create a new column for ordered hours (12pm to 12pm)
features_hourly['hour_ordered'] = features_hourly['hour'].apply(lambda x: x if x >= 12 else x + 24)

# Separate the data based on SBP (hypertensive vs. non-hypertensive)
hypertensive = features_hourly[features_hourly['SBP'] == 1]
non_hypertensive = features_hourly[features_hourly['SBP'] == 0]

feature = "Tsp"
for feature in basic_features:
    # Calculate the mean and standard error of Tsp for each hour of the day in the new order for both groups
    mean_tsp_hyper = hypertensive.groupby('hour_ordered')[feature].median().sort_index()
    sem_tsp_hyper = hypertensive.groupby('hour_ordered')[feature].sem().sort_index()

    mean_tsp_non_hyper = non_hypertensive.groupby('hour_ordered')[feature].median().sort_index()
    sem_tsp_non_hyper = non_hypertensive.groupby('hour_ordered')[feature].sem().sort_index()

    # Adjust the x-axis labels to match the new order
    ordered_hours_labels = [f"{hour % 24}:00" for hour in mean_tsp_hyper.index]
    plt.figure(figsize=(15, 10))
    plt.errorbar(range(len(mean_tsp_hyper)), mean_tsp_hyper.values, yerr=sem_tsp_hyper.values, fmt='-o', capsize=5, label='High BP')
    plt.errorbar(range(len(mean_tsp_non_hyper)), mean_tsp_non_hyper.values, yerr=sem_tsp_non_hyper.values, color = "r", fmt='-o', capsize=5, label='Low BP')
    # plt.axhline(y=Tsp_med_high, color='blue', linestyle='--', label='LAB Low BP')
    # plt.axhline(y=Tsp_med_low, color='red', linestyle='--', label='LAB High BP')
    plt.xlabel('Hour of the Day')
    plt.ylabel(feature)
    plt.title(f'{feature} per Hour of the Day (12pm to 12pm)')
    plt.grid(True)
    plt.xticks(range(len(mean_tsp_hyper)), ordered_hours_labels, rotation=45)
    plt.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)
    plt.show()

#### Plot

In [18]:
# Plot the mean Tsp with error bars representing the standard error for both groups
plt.figure(figsize=(15, 10))
plt.errorbar(range(len(mean_tsp_hyper)), mean_tsp_hyper.values, yerr=sem_tsp_hyper.values, fmt='-o', capsize=5, label='High BP')
plt.errorbar(range(len(mean_tsp_non_hyper)), mean_tsp_non_hyper.values, yerr=sem_tsp_non_hyper.values, color = "r", fmt='-o', capsize=5, label='Low BP')
# plt.axhline(y=Tsp_med_high, color='blue', linestyle='--', label='LAB Low BP')
# plt.axhline(y=Tsp_med_low, color='red', linestyle='--', label='LAB High BP')
plt.xlabel('Hour of the Day')
plt.ylabel(feature)
plt.title(f'{feature} per Hour of the Day (12pm to 12pm)')
plt.grid(True)
plt.xticks(range(len(mean_tsp_hyper)), ordered_hours_labels, rotation=45)
plt.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)
plt.show()

## Diagnostic features

In [12]:
data_path = '/Volumes/Untitled/HOM3ostasis/CinC/Data_24hPulses/mod_newQuality/'
sub_names = sorted([f for f in os.listdir(data_path) if not f.startswith('.')])

features_day = pd.DataFrame()
features_2min30s = pd.DataFrame()
features_hourly_diag = pd.DataFrame()

for i, sub_ID in enumerate(sub_names): 

    features_sub = pd.read_csv(data_path + sub_ID + '/features_diag_ALL.csv')
    features_sub["h/dT"] = h[sub_ID] / features_sub["dT"]
    # features_sub['sub_ID'] = sub_ID
    features_sub.index = pd.to_datetime(features_sub["start_datetime"])
    features_sub.drop(columns = "start_datetime", inplace = True)

    features_sub_2min30s = features_sub.resample("2.5 min").median()

    features_sub_hourly = features_sub.resample("h").median()

    # plt.figure(figsize=(15, 10))
    # plt.plot(features_sub_2min30s["Tsp"], label = "Original")
    # plt.plot(features_sub_hourly["Tsp"], label = "Hourly")
    # plt.plot(features_sub["sleep"] * 50)
    # plt.title(sub_ID, fontsize = 20)
    features_sub['sub_ID'] = sub_ID
    features_sub_2min30s["sub_ID"] = sub_ID
    features_sub_hourly["sub_ID"] = sub_ID

    # Add SBP information
    features_sub["SBP"] = SBP.loc[sub_ID].values[0]
    features_sub_2min30s["SBP"] = SBP.loc[sub_ID].values[0]
    features_sub_hourly["SBP"] = SBP.loc[sub_ID].values[0]

    features_day = pd.concat([features_day, features_sub], axis = 0)
    features_2min30s = pd.concat([features_2min30s, features_sub_2min30s], axis = 0)
    features_hourly_diag = pd.concat([features_hourly_diag, features_sub_hourly], axis = 0)
    # features_day = pd.concat([features_day, features_sub], axis = 0)

In [13]:
features_hourly_diag.drop(columns = [basic_features[0], basic_features[2], basic_features[3], basic_features[4], basic_features[5], basic_features[6]], inplace = True)

In [68]:
diagnostic_features = list(features_hourly_diag.columns)[6:]

In [69]:
# Convert the 'start_datetime' column to datetime format
features_hourly_diag['start_datetime'] = pd.to_datetime(features_hourly_diag.index)
features_hourly_diag.reset_index(drop = True, inplace = True)

# Extract the hour from the 'start_datetime' column
features_hourly_diag['hour'] = features_hourly_diag['start_datetime'].dt.hour

# Create a new column for ordered hours (12pm to 12pm)
features_hourly_diag['hour_ordered'] = features_hourly_diag['hour'].apply(lambda x: x if x >= 12 else x + 24)

# Separate the data based on SBP (hypertensive vs. non-hypertensive)
hypertensive = features_hourly_diag[features_hourly_diag['SBP'] == 1]
non_hypertensive = features_hourly_diag[features_hourly_diag['SBP'] == 0]

for feature in diagnostic_features:
    # Calculate the mean and standard error of Tsp for each hour of the day in the new order for both groups
    mean_hdT_hyper = hypertensive.groupby('hour_ordered')[feature].mean().sort_index()
    sem_hdT_hyper = hypertensive.groupby('hour_ordered')[feature].sem().sort_index()

    mean_hdT_non_hyper = non_hypertensive.groupby('hour_ordered')[feature].mean().sort_index()
    sem_hdT_non_hyper = non_hypertensive.groupby('hour_ordered')[feature].sem().sort_index()

    # Adjust the x-axis labels to match the new order
    ordered_hours_labels = [f"{hour % 24}:00" for hour in mean_hdT_hyper.index]
    plt.figure(figsize=(15, 10))
    plt.errorbar(range(len(mean_hdT_hyper)), mean_hdT_hyper.values, yerr=sem_hdT_hyper.values, fmt='-o', capsize=5, label='High BP')
    plt.errorbar(range(len(mean_hdT_non_hyper)), mean_hdT_non_hyper.values, yerr=sem_hdT_non_hyper.values, color = "r", fmt='-o', capsize=5, label='Low BP')
    # plt.axhline(y=hdT_med_high, color='blue', linestyle='--', label='LAB Low BP')
    # plt.axhline(y=hdT_med_low, color='red', linestyle='--', label='LAB High BP')
    plt.xlabel('Hour of the Day')
    plt.ylabel(feature)
    plt.title(f'{feature} per Hour of the Day (12pm to 12pm)')
    plt.grid(True)
    plt.xticks(range(len(mean_hdT_hyper)), ordered_hours_labels, rotation=45)
    plt.legend()
    plt.show()

#### h/dT

In [17]:
# Plot the mean h/dT with error bars representing the standard error for both groups
plt.figure(figsize=(15, 10))
plt.errorbar(range(len(mean_hdT_hyper)), mean_hdT_hyper.values, yerr=sem_hdT_hyper.values, fmt='-o', capsize=5, label='High BP')
plt.errorbar(range(len(mean_hdT_non_hyper)), mean_hdT_non_hyper.values, yerr=sem_hdT_non_hyper.values, color = "r", fmt='-o', capsize=5, label='Low BP')
plt.axhline(y=hdT_med_high, color='blue', linestyle='--', label='LAB Low BP')
plt.axhline(y=hdT_med_low, color='red', linestyle='--', label='LAB High BP')
plt.xlabel('Hour of the Day')
plt.ylabel(feature)
plt.title(f'{feature} per Hour of the Day (12pm to 12pm)')
plt.grid(True)
plt.xticks(range(len(mean_hdT_hyper)), ordered_hours_labels, rotation=45)
plt.legend()
plt.show()

#### IPA

In [107]:
# Plot the mean IPA with error bars representing the standard error for both groups
plt.figure(figsize=(15, 10))
plt.errorbar(range(len(mean_IPA_hyper)), mean_IPA_hyper.values, yerr=sem_IPA_hyper.values, fmt='-o', capsize=5, label='High BP')
plt.errorbar(range(len(mean_IPA_non_hyper)), mean_IPA_non_hyper.values, yerr=sem_IPA_non_hyper.values, color = "r", fmt='-o', capsize=5, label='Low BP')
# plt.axhline(y=IPA_med_high, color='blue', linestyle='--', label='LAB Low BP')
# plt.axhline(y=IPA_med_low, color='red', linestyle='--', label='LAB High BP')
plt.xlabel('Hour of the Day')
plt.ylabel('IPA')
plt.title('IPA per Hour of the Day (12pm to 12pm)')
plt.grid(True)
plt.xticks(range(len(mean_IPA_hyper)), ordered_hours_labels, rotation=45)
plt.legend()
plt.show()

## Merge hourly basic and diagnostic 

In [14]:
features_hourly["start_datetime"] = pd.to_datetime(features_hourly.index)
features_hourly.reset_index(drop = True, inplace = True)

features_hourly_diag["start_datetime"] = pd.to_datetime(features_hourly_diag.index)
features_hourly_diag.reset_index(drop = True, inplace = True)

In [81]:
# Merge hourly basic and diagnostic features

features_hourly_all = pd.concat([features_hourly.drop(columns = ["SBP", "Tpi", "Tsp/Tpi"]), features_hourly_diag.drop(columns = ["start_datetime", "sub_ID", "sleep"])], axis = 1)

In [72]:
features_hourly_all.columns

Index(['Tsp', 'Asp', 'AUCpi', 'Tsp/Asp', 'sleep', 'Tu', 'sub_ID',
       'start_datetime', 'Tsys', 'Tdia', 'Tdp', 'dT', 'Adn', 'Adp', 'AUCsys',
       'AUCdia', 'Tsys/Tdia', 'Adp/Asp', 'IPA', 'Asp/dT', 'Asp/(Tpi-Tsp)',
       'Rslope', 'Tv', 'Ta', 'Tb', 'h/dT', 'SBP'],
      dtype='object')

In [82]:
features_hourly_all['hour'] = features_hourly_all['start_datetime'].dt.hour

# Remove duplicate hours (some subjects have more than 24h of recording)
features_hourly_all.drop_duplicates(subset = ["sub_ID", "hour"], inplace = True) # keep first occurrence is default

# Create a new column for ordered hours (12pm to 12pm)
features_hourly_all['hour_ordered'] = features_hourly_all['hour'].apply(lambda x: x if x >= 12 else x + 24)

# Separate the data based on SBP (hypertensive vs. non-hypertensive)
hypertensive = features_hourly_all[features_hourly_all['SBP'] == 1]
non_hypertensive = features_hourly_all[features_hourly_all['SBP'] == 0]

features_list = list(features_hourly_all.columns)
features_list = ["Tsp"]

for feature in features_list:
    # Calculate the mean and standard error of Tsp for each hour of the day in the new order for both groups
    mean_hdT_hyper = hypertensive.groupby('hour_ordered')[feature].mean().sort_index() # this takes the mean of the values for each hour of the day
    sem_hdT_hyper = hypertensive.groupby('hour_ordered')[feature].sem().sort_index()

    mean_hdT_non_hyper = non_hypertensive.groupby('hour_ordered')[feature].mean().sort_index()
    sem_hdT_non_hyper = non_hypertensive.groupby('hour_ordered')[feature].sem().sort_index()

    # Adjust the x-axis labels to match the new order
    ordered_hours_labels = [f"{hour % 24}:00" for hour in mean_hdT_hyper.index]
    plt.figure(figsize=(15, 10))
    plt.errorbar(range(len(mean_hdT_hyper)), mean_hdT_hyper.values, yerr=sem_hdT_hyper.values, fmt='-o', color = "b", capsize=5, label='High BP')
    plt.errorbar(range(len(mean_hdT_non_hyper)), mean_hdT_non_hyper.values, yerr=sem_hdT_non_hyper.values, color = "r", fmt='-o', capsize=5, label='Low BP')
    # plt.axhline(y=hdT_med_high, color='blue', linestyle='--', label='LAB Low BP')
    # plt.axhline(y=hdT_med_low, color='red', linestyle='--', label='LAB High BP')
    plt.xlabel('Hour of the Day')
    plt.ylabel(feature)
    plt.title(f'{feature} per Hour of the Day (12pm to 12pm)')
    plt.grid(True)
    plt.xticks(range(len(mean_hdT_hyper)), ordered_hours_labels, rotation=45)
    plt.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
    plt.show()

: 

In [74]:
features_hourly_all.index = features_hourly_all["sub_ID"]
features_hourly_all.drop(columns = ["sub_ID"], inplace = True)
features_hourly_all

,Tsp,Asp,AUCpi,Tsp/Asp,sleep,Tu,start_datetime,Tsys,Tdia,Tdp,...,Asp/dT,Asp/(Tpi-Tsp),Rslope,Tv,Ta,Tb,h/dT,SBP,hour,hour_ordered
sub_ID,,,,,,,,,,,,,,,,,,,,,
10_AC,35.0,80.175455,149.705556,0.445545,0.0,17.0,2019-11-19 10:00:00,45.5,53.5,49.0,...,4.079928,0.968891,1.369220,64.0,15.0,36.0,10.250000,1.0,10,34
10_AC,37.0,53.165808,146.334015,0.694220,0.0,16.0,2019-11-19 11:00:00,52.5,46.5,58.5,...,3.014997,0.773260,0.981345,56.0,13.0,37.0,7.321429,1.0,11,35
10_AC,35.0,57.491111,145.692424,0.617666,0.0,15.0,2019-11-19 12:00:00,46.5,52.5,54.0,...,2.678902,0.720081,1.003144,64.0,13.0,36.0,8.415789,1.0,12,12
10_AC,37.0,61.668182,182.792929,0.603633,0.0,16.0,2019-11-19 13:00:00,51.0,48.0,54.5,...,3.109345,0.970003,1.178161,64.0,14.0,36.5,8.004762,1.0,13,13
10_AC,36.0,76.399697,147.240429,0.471683,0.0,15.0,2019-11-19 14:00:00,45.0,54.0,48.0,...,3.770833,0.959747,1.287363,64.0,14.0,34.0,10.933333,1.0,14,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9_BC,30.0,127.418182,185.759343,0.239945,0.0,10.0,2019-11-16 06:00:00,45.0,54.0,50.0,...,3.366067,0.989968,1.543135,55.5,10.5,31.0,7.287785,0.0,6,30
9_BC,28.0,105.776364,152.861768,0.270921,0.0,10.0,2019-11-16 07:00:00,41.0,58.0,46.5,...,5.466882,1.469064,2.258902,54.5,11.0,33.0,8.916409,0.0,7,31
9_BC,27.0,234.726364,277.886061,0.095766,0.0,10.0,2019-11-16 08:00:00,42.0,57.0,47.0,...,4.864366,1.250837,2.084728,50.0,9.0,33.0,8.421053,0.0,8,32


In [78]:
# features_hourly_all.drop(columns = ["start_datetime", "hour_ordered"], inplace = True)

# put sleep column as second to last
sleep = features_hourly_all["sleep"]
features_hourly_all.drop(columns = ["sleep"], inplace = True)
features_hourly_all["sleep"] = sleep


In [79]:
features_hourly_all.to_csv('/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/Cinc/Data/paper/features_24h_hourly.csv')

Number of hours of recording per subject 

In [57]:
sns.set_context("talk")

plt.figure(figsize=(15, 10))
features_hourly_all.groupby(['sub_ID'])["Tsp"].count().plot(kind='bar')
plt.ylabel('Number of hours')
plt.title('Number of hours of recording per subject')
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Figures/num_hours_per_subject_24hours.png", dpi = 300, bbox_inches = 'tight')

In [48]:
# drop duplicates hour for each subject
features_hourly_all.drop_duplicates(subset = ["sub_ID", "hour"], inplace = True)
plt.figure(figsize=(15, 10))
features_hourly_all.groupby(['sub_ID'])["Tsp"].count().plot(kind='bar')

<Axes: xlabel='sub_ID'>

In [50]:
# Separate the data based on SBP (hypertensive vs. non-hypertensive)
hypertensive = features_hourly_all[features_hourly_all['SBP'] == 1]
non_hypertensive = features_hourly_all[features_hourly_all['SBP'] == 0]

features_list = list(features_hourly_all.columns)
features_list = ["Tsp"]

for feature in features_list:
    # Calculate the mean and standard error of Tsp for each hour of the day in the new order for both groups
    mean_hdT_hyper = hypertensive.groupby('hour_ordered')[feature].mean().sort_index() # this takes the mean of the values for each hour of the day
    sem_hdT_hyper = hypertensive.groupby('hour_ordered')[feature].sem().sort_index()

    mean_hdT_non_hyper = non_hypertensive.groupby('hour_ordered')[feature].mean().sort_index()
    sem_hdT_non_hyper = non_hypertensive.groupby('hour_ordered')[feature].sem().sort_index()

    # Adjust the x-axis labels to match the new order
    ordered_hours_labels = [f"{hour % 24}:00" for hour in mean_hdT_hyper.index]
    plt.figure(figsize=(15, 10))
    plt.errorbar(range(len(mean_hdT_hyper)), mean_hdT_hyper.values, yerr=sem_hdT_hyper.values, fmt='-o', color = "b", capsize=5, label='High BP')
    plt.errorbar(range(len(mean_hdT_non_hyper)), mean_hdT_non_hyper.values, yerr=sem_hdT_non_hyper.values, color = "r", fmt='-o', capsize=5, label='Low BP')
    # plt.axhline(y=hdT_med_high, color='blue', linestyle='--', label='LAB Low BP')
    # plt.axhline(y=hdT_med_low, color='red', linestyle='--', label='LAB High BP')
    plt.xlabel('Hour of the Day')
    plt.ylabel(feature)
    plt.title(f'{feature} per Hour of the Day (12pm to 12pm)')
    plt.grid(True)
    plt.xticks(range(len(mean_hdT_hyper)), ordered_hours_labels, rotation=45)
    plt.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=24)
    plt.show()

# Single Values -- Real-World

#### Basic
I wll combine them later with diagnostic

In [143]:
import warnings
warnings.filterwarnings("ignore")

In [166]:
data_path = '/Volumes/Untitled/HOM3ostasis/CinC/Data_24hPulses/mod_newQuality/'
sub_names = sorted([f for f in os.listdir(data_path) if not f.startswith('.')])

features_day = pd.DataFrame()
features_2min30s = pd.DataFrame()
features_realWorld_wake_2min30s = pd.DataFrame()
features_realWorld_sleep_2min30s = pd.DataFrame()

for i, sub_ID in enumerate(sub_names): 

    features_sub = pd.read_csv(data_path + sub_ID + '/features_basic_ALL.csv')
    features_sub.index = pd.to_datetime(features_sub["start_datetime"])
    features_sub.drop(columns = "start_datetime", inplace = True)
    fiducials = pd.read_csv(data_path + sub_ID + '/fiducials_basic_ALL.csv').drop(columns = "Unnamed: 0")
    fiducials.index = pd.to_datetime(fiducials["start_datetime"])
    fiducials.drop(columns = "start_datetime", inplace = True)
    features_sub["Tu"] = fiducials["u"] - fiducials["onset"]

    features_sub_2min30s = features_sub.resample("2.5 min").median()

    features_sub_realWorld_wake_2min30s = features_sub_2min30s[features_sub_2min30s["sleep"] == 0]
    features_sub_realWorld_sleep_2min30s = features_sub_2min30s[features_sub_2min30s["sleep"] == 1]

    features_sub['sub_ID'] = sub_ID
    features_sub_2min30s["sub_ID"] = sub_ID
    # features_sub_realWorld_wake_2min30s["sub_ID"] = sub_ID
    features_sub_realWorld_wake_2min30s.loc[:, "sub_ID"] = sub_ID
    # features_sub_realWorld_sleep_2min30s["sub_ID"] = sub_ID
    features_sub_realWorld_sleep_2min30s.loc[:, "sub_ID"] = sub_ID

    # Add SBP information
    features_sub["SBP"] = SBP.loc[sub_ID].values[0]
    features_sub_2min30s["SBP"] = SBP.loc[sub_ID].values[0]
    # features_sub_realWorld_wake_2min30s["SBP"] = SBP.loc[sub_ID].values[0]
    features_sub_realWorld_wake_2min30s.loc[:, "SBP"] = SBP.loc[sub_ID].values[0]
    # features_sub_realWorld_sleep_2min30s["SBP"] = SBP.loc[sub_ID].values[0]
    features_sub_realWorld_sleep_2min30s.loc[:, "SBP"] = SBP.loc[sub_ID].values[0]

    features_day = pd.concat([features_day, features_sub], axis = 0)
    features_2min30s = pd.concat([features_2min30s, features_sub_2min30s], axis = 0)
    features_realWorld_wake_2min30s = pd.concat([features_realWorld_wake_2min30s, features_sub_realWorld_wake_2min30s], axis = 0)
    features_realWorld_sleep_2min30s = pd.concat([features_realWorld_sleep_2min30s, features_sub_realWorld_sleep_2min30s], axis = 0)

In [167]:
features_realWorld_wake_2min30s.drop(columns = ["Tpi", "Tsp/Tpi", "sleep"], inplace = True)
features_realWorld_sleep_2min30s.drop(columns = ["Tpi", "Tsp/Tpi", "sleep"], inplace = True)

In [168]:
features_realWorld_wake = features_realWorld_wake_2min30s.groupby("sub_ID").median()
features_realWorld_sleep = features_realWorld_sleep_2min30s.groupby("sub_ID").median()

features_realWorld_wake.loc[features_realWorld_wake["SBP"] == 0, "SBP"] = "Low BP"
features_realWorld_wake.loc[features_realWorld_wake["SBP"] == 1, "SBP"] = "High BP"

features_realWorld_sleep.loc[features_realWorld_sleep["SBP"] == 0, "SBP"] = "Low BP"
features_realWorld_sleep.loc[features_realWorld_sleep["SBP"] == 1, "SBP"] = "High BP"

In [149]:
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('talk')

#### Diagnostic

In [172]:
data_path = '/Volumes/Untitled/HOM3ostasis/CinC/Data_24hPulses/mod_newQuality/'
sub_names = sorted([f for f in os.listdir(data_path) if not f.startswith('.')])

features_day = pd.DataFrame()
features_2min30s = pd.DataFrame()
features_realWorld_wake_2min30s_diag = pd.DataFrame()
features_realWorld_sleep_2min30s_diag = pd.DataFrame()

for i, sub_ID in enumerate(sub_names): 

    features_sub = pd.read_csv(data_path + sub_ID + '/features_diag_ALL.csv')
    features_sub["h/dT"] = h[sub_ID] / features_sub["dT"]
    features_sub.index = pd.to_datetime(features_sub["start_datetime"])
    features_sub.drop(columns = "start_datetime", inplace = True)

    features_sub_2min30s = features_sub.resample("2.5 min").median()

    features_sub_realWorld_wake_2min30s = features_sub_2min30s[features_sub_2min30s["sleep"] == 0]
    features_sub_realWorld_sleep_2min30s = features_sub_2min30s[features_sub_2min30s["sleep"] == 1]

    features_sub['sub_ID'] = sub_ID
    features_sub_2min30s["sub_ID"] = sub_ID
    features_sub_realWorld_wake_2min30s["sub_ID"] = sub_ID
    features_sub_realWorld_sleep_2min30s["sub_ID"] = sub_ID

    # Add SBP information
    features_sub["SBP"] = SBP.loc[sub_ID].values[0]
    features_sub_2min30s["SBP"] = SBP.loc[sub_ID].values[0]
    features_sub_realWorld_wake_2min30s["SBP"] = SBP.loc[sub_ID].values[0]
    features_sub_realWorld_sleep_2min30s["SBP"] = SBP.loc[sub_ID].values[0]

    features_day = pd.concat([features_day, features_sub], axis = 0)
    features_2min30s = pd.concat([features_2min30s, features_sub_2min30s], axis = 0)
    features_realWorld_wake_2min30s_diag = pd.concat([features_realWorld_wake_2min30s_diag, features_sub_realWorld_wake_2min30s], axis = 0)
    features_realWorld_sleep_2min30s_diag = pd.concat([features_realWorld_sleep_2min30s_diag, features_sub_realWorld_sleep_2min30s], axis = 0)

In [173]:
features_realWorld_wake_2min30s_diag.drop(columns = ["Tsp/Tpi", "sleep"], inplace = True)
features_realWorld_sleep_2min30s_diag.drop(columns = ["Tsp/Tpi", "sleep"], inplace = True)
features_realWorld_wake_2min30s_diag.drop(columns = [basic_features[0], basic_features[2], basic_features[3], basic_features[5], basic_features[6]], inplace = True)
features_realWorld_sleep_2min30s_diag.drop(columns = [basic_features[0], basic_features[2], basic_features[3], basic_features[5], basic_features[6]], inplace = True)

In [174]:
features_realWorld_wake_diag = features_realWorld_wake_2min30s_diag.groupby("sub_ID").median()
features_realWorld_sleep_diag = features_realWorld_sleep_2min30s_diag.groupby("sub_ID").median()

features_realWorld_wake_diag.loc[features_realWorld_wake_diag["SBP"] == 0, "SBP"] = "Low BP"
features_realWorld_wake_diag.loc[features_realWorld_wake_diag["SBP"] == 1, "SBP"] = "High BP"

features_realWorld_sleep_diag.loc[features_realWorld_sleep_diag["SBP"] == 0, "SBP"] = "Low BP"
features_realWorld_sleep_diag.loc[features_realWorld_sleep_diag["SBP"] == 1, "SBP"] = "High BP"

### Merge basic and diagnostic

In [177]:
# Merge basic and diagnostic features
features_realWorld_wake_ALL = features_realWorld_wake.drop(columns="SBP").merge(features_realWorld_wake_diag, left_index = True, right_index = True)
features_realWorld_sleep_ALL = features_realWorld_sleep.drop(columns="SBP").merge(features_realWorld_sleep_diag, left_index = True, right_index = True)

features_realWorld_wake_ALL.head()

,Tsp,Asp,AUCpi,Tsp/Asp,Tu,Tsys,Tdia,Tdp,dT,Adn,...,Adp/Asp,IPA,Asp/dT,Asp/(Tpi-Tsp),Rslope,Tv,Ta,Tb,h/dT,SBP
sub_ID,,,,,,,,,,,,,,,,,,,,,
10_AC,35.0,68.911818,192.172386,0.511953,15.0,47.0,52.0,54.0,20.5,55.005455,...,0.878388,0.062901,3.458271,1.010938,1.409321,60.00,13.5,36.0,8.200000,High BP
11_EC,37.0,76.520000,172.209091,0.487857,11.0,44.0,55.0,45.0,15.0,65.474167,...,0.940537,-0.524790,5.764074,1.137558,1.755874,58.00,9.0,32.0,11.214286,High BP
12_FN,33.5,53.486313,210.547778,0.666554,13.0,46.0,53.0,55.0,24.0,42.120000,...,0.823890,-0.061708,2.654387,0.841625,1.183939,48.00,10.5,34.0,7.469697,Low BP
13_MS,34.0,81.811111,258.487702,0.424763,12.5,45.0,54.0,51.0,20.0,62.514444,...,0.874277,-0.372126,4.384924,1.151790,1.709645,57.50,10.0,31.0,8.000000,Low BP
14_AD,32.0,81.213662,168.233636,0.403579,13.0,45.5,53.5,54.0,27.0,52.116515,...,0.674272,-0.661965,3.283433,1.183819,1.845802,37.75,10.5,35.0,6.407407,Low BP


In [181]:
feature = "h/dT"

plt.figure(figsize=(15, 10))
plt.subplot(1, 3, 1)
sns.boxplot(data = features_lab_median, x = "SBP", y = feature, hue = "SBP", showfliers = True)
plt.subplot(1, 3, 2, sharey = plt.subplot(1, 3, 1))
sns.boxplot(data = features_realWorld_wake_ALL, x = "SBP", hue = "SBP", y = feature, showfliers = True)
plt.subplot(1, 3, 3, sharey = plt.subplot(1, 3, 1))
sns.boxplot(data = features_realWorld_sleep_ALL, x = "SBP", hue = "SBP", y = feature, showfliers = True)
plt.tight_layout()

### Save

In [196]:
features_lab_median.drop(columns = ["Tpi", "Tsp/Tpi"], inplace = True)

In [197]:
features_lab_median.shape, features_realWorld_wake_ALL.shape, features_realWorld_sleep_ALL.shape

((39, 24), (39, 24), (39, 24))

In [199]:
# order the columns in features_realWorld_wake_ALL and in features_realWorld_sleep_ALL as they are in features_lab_median

features_realWorld_wake_ALL = features_realWorld_wake_ALL[features_lab_median.columns]
features_realWorld_sleep_ALL = features_realWorld_sleep_ALL[features_lab_median.columns]

In [200]:
features_realWorld_wake_ALL.to_csv('/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/Cinc/Data/paper/features_realWorld_WAKE.csv')
features_realWorld_sleep_ALL.to_csv('/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/Cinc/Data/paper/features_realWorld_SLEEP.csv')
features_lab_median.to_csv('/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/Cinc/Data/paper/features_lab.csv')

In [203]:
feature = "Tu"

plt.figure(figsize=(15, 10))
plt.subplot(1, 3, 1)
sns.boxplot(data = features_lab_median, x = "SBP", y = feature, hue = "SBP", showfliers = True)
plt.subplot(1, 3, 2, sharey = plt.subplot(1, 3, 1))
sns.boxplot(data = features_realWorld_wake_ALL, x = "SBP", hue = "SBP", y = feature, showfliers = True)
plt.subplot(1, 3, 3, sharey = plt.subplot(1, 3, 1))
sns.boxplot(data = features_realWorld_sleep_ALL, x = "SBP", hue = "SBP", y = feature, showfliers = True)
plt.tight_layout()

In [65]:
import statsmodels.stats.power as smp

# Parameters for the power analysis
effect_size = 0.25  # small effect size (Cohen's f)
alpha = 0.05        # significance level
power = 0.8       # desired power
k = 3               # number of groups

# Performing the power analysis
analysis = smp.FTestAnovaPower()
sample_size = analysis.solve_power(effect_size=effect_size, alpha=alpha, power=power, k_groups=k)

print(f"Required sample size per group for three-way ANOVA: {sample_size}")

Required sample size per group for three-way ANOVA: 157.189792421322
